<a href="https://colab.research.google.com/github/sarang-manohar/Coursera_Capstone/blob/main/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook puts in the seed of the Capstone Project with Coursera!

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder
import folium
from pandas.io.json import json_normalize
print('Libraries imported')

Libraries imported


In [2]:
html_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

soup = BeautifulSoup(html_data.text, 'html.parser')
table = soup.find("table",{"rules":"all"})

table_data = []

for row in table.find_all('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_data.append(cell)

df=pd.DataFrame(table_data)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df['Address'] = df['Borough']+', Toronto, Ontario, Canada, '+df['PostalCode']
df.head()

,PostalCode,Borough,Neighborhood,Address
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A"
1,M4A,North York,Victoria Village,"North York, Toronto, Ontario, Canada, M4A"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Downtown Toronto, Toronto, Ontario, Canada, M5A"
3,M6A,North York,"Lawrence Manor, Lawrence Heights","North York, Toronto, Ontario, Canada, M6A"
4,M7A,Queen's Park,Ontario Provincial Government,"Queen's Park, Toronto, Ontario, Canada, M7A"


In [3]:
lat=[]
long=[]
for i, pc in enumerate(df['PostalCode']):
    with requests.Session() as session:
        g = geocoder.arcgis('{}, Toronto, Canada'.format(pc),session=session)
    lat.append(g.latlng[0])
    long.append(g.latlng[1])

df['Latitude'] = lat
df['Longitude'] = long

df.head()

,PostalCode,Borough,Neighborhood,Address,Latitude,Longitude
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A",43.75245,-79.32991
1,M4A,North York,Victoria Village,"North York, Toronto, Ontario, Canada, M4A",43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Downtown Toronto, Toronto, Ontario, Canada, M5A",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights","North York, Toronto, Ontario, Canada, M6A",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,"Queen's Park, Toronto, Ontario, Canada, M7A",43.66253,-79.39188


In [4]:
address = 'Toronto, Ontario, Canada'

g = geocoder.arcgis(address)
latitude = g.latlng[0]
longitude = g.latlng[1]
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.648690000000045, -79.38543999999996.


In [5]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto) 

map_Toronto

In [6]:
CLIENT_ID = 'Q5LJU1V0DPCB0R5XDREYLILRZGTSPSBFPBX1QGAQHZAAPNPK' # your Foursquare ID
CLIENT_SECRET = 'U4XJXABOBZBSHOWMLEQYHNEYVSXAR0VBAYAUGHLBBQ3LJIVX' # your Foursquare Secret
VERSION = '20210704' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
ACCESS_TOKEN = '0HUXGDI5F3EDNXPW3RW21KJNV4TB3Z4ITMA34M5VRKXR0HRA'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Q5LJU1V0DPCB0R5XDREYLILRZGTSPSBFPBX1QGAQHZAAPNPK
CLIENT_SECRET:U4XJXABOBZBSHOWMLEQYHNEYVSXAR0VBAYAUGHLBBQ3LJIVX


In [7]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&GET'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION)
results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.drop({'categories'},axis=1,inplace=True)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

<ipython-input-13-54553548249f>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


In [15]:
df.head(1)

,PostalCode,Borough,Neighborhood,Address,Latitude,Longitude
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A",43.75245,-79.32991


In [41]:
df_test = df.copy()
df_test = df_test.head(2)
df_test

,PostalCode,Borough,Neighborhood,Address,Latitude,Longitude
0,M3A,North York,Parkwoods,"North York, Toronto, Ontario, Canada, M3A",43.75245,-79.32991
1,M4A,North York,Victoria Village,"North York, Toronto, Ontario, Canada, M4A",43.73057,-79.31306


In [110]:
venues_list=[]
for name, lat, lng in zip(df['Neighborhood'], df['Latitude'], df['Longitude']):
    print(name)
        
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        500, 
        LIMIT)
        
    # make the GET request
    results = requests.get(url).json()['response']['groups'][0]['items']
    # return only relevant information for each nearby venue
    venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    # return only relevant information for each nearby venue
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood', 
              'Neighborhood Latitude', 
              'Neighborhood Longitude', 
              'Venue', 
              'Venue Latitude', 
              'Venue Longitude', 
              'Venue Category']

nearby_venues.sample(10)


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
478,Central Bay Street,43.65609,-79.38493,Japango,43.655268,-79.385165,Sushi Restaurant
1956,Enclave of M5E,43.64869,-79.38544,Royal Alexandra Theatre,43.647255,-79.387530,Theater
1598,Agincourt,43.79452,-79.26708,Canton Kitchen 廣東小炒,43.797772,-79.270981,Chinese Restaurant
125,"Garden District, Ryerson",43.65739,-79.37804,Jazz Bistro,43.655678,-79.379276,Music Venue
424,Berczy Park,43.64536,-79.37306,Garrison Bespoke,43.648102,-79.376334,Tailor Shop
614,"Richmond, Adelaide, King",43.64970,-79.38258,GoodLife Fitness Toronto 137 Yonge Street,43.651242,-79.378068,Gym
62,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042,AllSaints,43.724378,-79.453646,Clothing Store
697,"Fairview, Henry Farm, Oriole",43.78097,-79.34781,McDonald's,43.778407,-79.343574,Fast Food Restaurant
607,"Richmond, Adelaide, King",43.64970,-79.38258,Tim Hortons,43.646862,-79.382544,Coffee Shop
803,"Little Portugal, Trinity",43.64848,-79.41774,Pizzeria Libretto,43.648979,-79.420604,Pizza Place
